In [1]:
import pandas as pd
import numpy as np
#we want to give highest weightage 50 
#second is 30 
#third is 20

In [ ]:
#here we need to add weights instead of multiplying by 1/3 and those weighst are calculated by the probability in the table
#for highest probability give weight 50 
#for 2nd highest give 30 
#for lowest give 20
def balanced_profolio(target, pred):
    #target_ = target.iloc[:, 1:4]
    target = target.iloc[:, -3:]
    start = 100000
    initial = 100000
    s_p = target.iloc[:, 4]#add this 
    #real = target.iloc[:, -3:]
    real = target.iloc[:, 1:4]
    compare_result = []
    for i in range(pred.shape[0]):
        row = target_.iloc[i, :]
        start = sum(list(map(lambda x: (start / 3) * (x), row)))#here we do start *(.5/.3/.2 based on weight)*x
        if pred.iloc[i,:].sum() >= 2:
            op_invest = 1
            op_s_p = -1
        else:
            op_invest = -1
            op_s_p = 1
        if real.iloc[i, :].sum() >= 2:
            op_real = 1
        else:
            op_real = -1
        if op_real != op_invest:
            op_final = -1
        else:
            op_final = 1
        result_pred = (op_invest * start + op_s_p * s_p[i] * initial) * op_final
        compare_result.append(result_pred)
    return compare_result